<h1><center>MNIST classification using Numpy<center></h1>

**NOM: EL MARCHOUK KENZA et BARRY SORY IBRAHIMA**

## Importing Numpy and Keras

In [34]:
import numpy as np
import tensorflow as tf
import keras
print("Using tensorflow version " + str(tf.__version__))
print("Using keras version " + str(keras.__version__))

Using tensorflow version 2.8.0
Using keras version 2.8.0


## Loading and preparing the MNIST dataset
Load the MNIST dataset made available by keras.datasets. Check the size of the training and testing sets. 

In [35]:
# The MNSIT dataset is ready to be imported from Keras into RAM
# Warning: you cannot do that for larger databases (e.g., ImageNet)
from keras.datasets import mnist
# START CODE HERE
data = mnist.load_data()
(X_train, y_train), (X_test, y_test) = data
print("the size of training set: ",X_train.shape,y_train.shape)
print("the size of testing sets: ", X_test.shape,y_test.shape)
# END CODE HERE
print(y_train.shape)

the size of training set:  (60000, 28, 28) (60000,)
the size of testing sets:  (10000, 28, 28) (10000,)
(60000,)


The MNIST database contains 60,000 training images and 10,000 testing images.
Using the pyplot package, visualize the first sample of the training set:

In [36]:
# Let us visualize the first training sample using the Matplotlib library
from matplotlib import pyplot as plt
# START CODE HERE
img_per_row =9
fig,ax = plt.subplots(nrows=2, ncols=img_per_row,
                      figsize=(18,4),
                      subplot_kw=dict(xticks=[], yticks=[]))
for row in [0, 1]:
    for col in range(img_per_row):
        ax[row, col].imshow(X_train[row*img_per_row + col].astype('int'))   
plt.show()
# END CODE HERE

The database contains images of handwritten digits. Hence, they belong to one of 10 categories, depending on the digit they represent. 
Reminder: in order to do multi-class classification, we use the softmax function, which outputs a multinomial probability distribution. That means that the output to our model will be a vector of size $10$, containing probabilities (meaning that the elements of the vector will be positive sum to $1$).
For easy computation, we want to true labels to be represented with the same format: that is what we call **one-hot encoding**. For example, if an image $\mathbf{x}$ represents the digit $5$, we have the corresponding one_hot label (careful, $0$ will be the first digit): 
$$ \mathbf{y} = [0, 0, 0, 0, 0, 1, 0, 0, 0, 0] $$
Here, you need to turn train and test labels to one-hot encoding using the following function: 

In [37]:
from keras.utils.np_utils import to_categorical
# START CODE HERE
# one hot encode
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

# END CODE HERE

Images are black and white, with size $28 \times 28$. We will work with them using a simple linear classification model, meaning that we will have them as vectors of size $(784)$.
You should then transform the images to the size $(784)$ using the numpy function ```reshape```.

Then, after casting the pixels to floats, normalize the images so that they have zero-mean and unitary deviation. Be careful to your methodology: while you have access to training data, you may not have access to testing data, and must avoid using any statistic on the testing dataset.

In [38]:

# Reshape to proper images with 1 color channel according to backend scheme
from keras.preprocessing.image import ImageDataGenerator
img_rows, img_cols = X_train.shape[1], X_train.shape[2]
X_train = X_train.reshape(-1, img_rows*img_cols)
X_test=X_test.reshape(len(X_test), -1)
# START CODE HERE
#train_iterator = datagen.flow(X_train, y_train, batch_size=64)
# END CODE HERE

# Cast pixels from uint8 to float32
train_images = X_train.astype('float32')
test_images=X_test.astype('float32')
# calculate global mean and standard deviation
mean, std = train_images.mean(), train_images.std()
mean1,std1= test_images.mean(), test_images.std()
print('Mean before: %.3f, Standard Deviation before: %.3f' % (mean, std))
print('Mean before test: %.3f, Standard Deviation before test: %.3f' % (mean1, std1))
# global standardization of pixels
train_images = (train_images - mean) / std
test_images = (test_images - mean1) / std1
# confirm it had the desired effect
mean, std = train_images.mean(), train_images.std()
mean1, std1 = test_images.mean(), test_images.std()
print('Mean: %.3f, Standard Deviation: %.3f' % (mean, std))
print('Mean est: %.3f, Standard Deviation test: %.3f' % (mean1, std1))
# Now let us normalize the images so that they have zero mean and standard deviation
# Hint: are real testing data statistics known at training time ?
# START CODE HERE
...
# END CODE HERE

Mean before: 33.318, Standard Deviation before: 78.567
Mean before test: 33.791, Standard Deviation before test: 79.172
Mean: -0.000, Standard Deviation: 1.000
Mean est: -0.000, Standard Deviation test: 1.000


Ellipsis

In [39]:
train_images.shape

(60000, 784)

# Working with Numpy

Look at this [cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf) for some basic information on how to use numpy.

## Defining the model 

We will here create a simple, linear classification model. We will take each pixel in the image as an input feature (making the size of the input to be $784$) and transform these features with a weight matrix $\mathbf{W}$ and a bias vector $\mathbf{b}$. Since there is $10$ possible classes, we want to obtain $10$ scores. Then, 
$$ \mathbf{W} \in \mathbb{R}^{784 \times 10} $$
$$ \mathbf{b} \in \mathbb{R}^{10} $$

and our scores are obtained with:
$$ \mathbf{z} = \mathbf{W}^{T} \mathbf{x} +  \mathbf{b} $$

where $\mathbf{x} \in \mathbb{R}^{784}$ is the input vector representing an image.
We note $\mathbf{y} \in \mathbb{R}^{10}$ as the target one_hot vector. 

Here, you fist need to initialize $\mathbf{W}$ and $\mathbf{b}$ using ```np.random.normal``` and ```np.zeros```, then compute $\mathbf{z}$.

In [40]:
# To avoid implementing a complicated gradient back-propagation,
# we will try a very simple architecture with one layer 
def initLayer(n_input,n_output):
    """
    Initialize the weights, return the number of parameters
    Inputs: n_input: the number of input units - int
          : n_output: the number of output units - int
    Outputs: W: a matrix of weights for the layer - numpy ndarray
           : b: a vector bias for the layer - numpy ndarray
           : nb_params: the number of parameters  - int
    """
    # START CODE HERE
    W=np.random.normal(0,1,size=(n_input, n_output))
    b = np.zeros(n_output)
    nb_params=n_input+n_output
    #nb_params=n_input*n_output + n_output
    return W, b, nb_params
    # END CODE HERE

In [41]:
n_training = train_images.shape[0] 
n_feature = train_images.shape[1] 
n_labels = 10
W, b, nb_params = initLayer(n_feature, n_labels)
#print('W shape:', W.shape)
#print('b shape:', b.shape)
#print('train images shape:', train_images.shape)

In [42]:
def forward(W, b, X):
    """
    Perform the forward propagation
    Inputs: W: the weights - numpy ndarray
          : b: the bias - numpy ndarray
          : X: the batch - numpy ndarray
    Outputs: z: outputs - numpy ndarray
    """
    #z = W.T@X + b
    #z= np.dot(W.transpose(),X) + b
    #z=X @ W + b 
    z= np.dot(X,W) + b
    return z

## Computing the output 

To obtain classification probabilities, we use the softmax function:
$$ \mathbf{o} = softmax(\mathbf{z}) \text{         with          } o_i = \frac{\exp(z_i)}{\sum_{j=0}^{9} \exp(z_j)} $$

The usual difficulty with the softmax function is the possibility of overflow when the scores $z_i$ are already large. Since a softmax is not affected by a shift affecting the whole vector $\mathbf{z}$:
$$ \frac{\exp(z_i - c)}{\sum_{j=0}^{9} \exp(z_j - c)} =  \frac{\exp(c) \exp(z_i)}{\exp(c) \sum_{j=0}^{9} \exp(z_j)} = \frac{\exp(z_i)}{\sum_{j=0}^{9} \exp(z_j)}$$
what trick can we use to ensure we will not encounter any overflow ? 

In [43]:
def softmax(z):
    """
    Perform the softmax transformation to the pre-activation values
    Inputs: z: the pre-activation values - numpy ndarray
    Outputs: out: the activation values - numpy ndarray
    """
    damping = np.max(z)
    z = z - damping
    out = np.exp(z)/np.sum(np.exp(z))
    return out

## Making updates

We define a learning rate $\eta$. The goal is to be able to apply updates:
$$ \mathbf{W}^{t+1} = \mathbf{W}^{t} - \nabla_{\mathbf{W}} l_{MLE} $$

In order to do this, we will compute this gradient (and the bias) in the function ```update```. In the next function ```updateParams```, we will actually apply the update with regularization. 

Reminder: the gradient $\nabla_{\mathbf{W}} l_{MLE}$ is the matrix containing the partial derivatives 
$$ \left[\frac{\delta l_{MLE}}{\delta W_{ij}}\right]_{i=1..784, j=1..10} $$
**Remark**: Careful, the usual way of implementing this in python has the dimensions of $\mathbf{W}$ reversed compared to the notation of the slides.

Coordinate by coordinate, we obtain the following update: 
$$ W_{ij}^{t+1} = W_{ij}^{t} - \eta \frac{\delta l_{MLE}}{\delta W_{ij}} $$

Via the chain rule, we obtain, for an input feature $i \in [0, 783]$ and a output class $j \in [0, 9]$: $$\frac{\delta l_{MLE}}{\delta W_{ij}} = \frac{\delta l_{MLE}}{\delta z_{j}} \frac{\delta z_j}{\delta W_{ij}}$$ 

It's easy to compute that $\frac{\delta z_j}{\delta W_{ij}} = x_i$

We compute the softmax derivative, to obtain:
$$ \nabla_{\mathbf{z}} l_{MLE} = \mathbf{o} - \mathbf{y} $$

Hence, $\frac{\delta l_{MLE}}{\delta z_{j}} = o_j - y_j$ and we obtain that $$\frac{\delta l_{MLE}}{\delta W_{ij}} = (o_j - y_j) x_i$$

This can easily be written as a scalar product, and a similar computation (even easier, actually) can be done for $\mathbf{b}$. Noting $\nabla_{\mathbf{z}} l_{MLE} = \mathbf{o} - \mathbf{y}$ as ```grad``` in the following function, compute the gradients $\nabla_{\mathbf{W}} l_{MLE}$ and $\nabla_{\mathbf{b}} l_{MLE}$ in order to call the function ```updateParams```.

Note: the regularizer and the weight_decay $\lambda$ are used in ```updateParams```.

In [44]:
def update(eta, W, b, grad, X, regularizer, weight_decay):
    """
    Perform the update of the parameters
    Inputs: eta: the step-size of the gradient descent - float 
          : W: the weights - ndarray
          : b: the bias -  ndarray
          : grad: the gradient of the activations w.r.t. to the loss -  list of ndarray
          : X: the data -  ndarray
          : regularizer: 'L2' or None - the regularizer to be used in updateParams
          : weight_decay: the weight decay to be used in updateParams - float
    Outputs: W: the weights updated -  ndarray
           : b: the bias updated -  ndarray
    """
    grad_w = X.reshape((-1,1)) @ grad.reshape((-1,1)).T
    grad_b = grad
        
    W = updateParams(W, grad_w, eta, regularizer, weight_decay)
    b = updateParams(b, grad_b, eta, regularizer, weight_decay)
    return W, b

In [45]:
train_images.reshape((-1,1)).shape

(47040000, 1)

The update rule is affected by regularization. We implement two cases: No regularization, or L2 regularization. Use the two possible update rules to implement the following function:

In [46]:
def updateParams(param, grad_param, eta, regularizer=None, weight_decay=0.):
    """
    Perform the update of the parameters
    Inputs: param: the network parameters - ndarray
          : grad_param: the updates of the parameters - ndarray
          : eta: the step-size of the gradient descent - float
          : weight_decay: the weight-decay - float
    Outputs: the parameters updated - ndarray
    """
    if regularizer==None:
        return param - eta*grad_param
    elif regularizer=='L2':
        return (1-2*weight_decay)*param - eta*grad_param
    else:
        raise NotImplementedError

## Computing the Accuracy

Here, we simply use the model to predict the class (by taking the argmax of the output !) for every example in ```X```, and count the number of times the model is right, to output the accuracy.

In [47]:
def computeAcc(W, b, X, labels):
    """
    Compute the loss value of the current network on the full batch
    Inputs: act_func: the activation function - function
          : W: the weights - list of ndarray
          : B: the bias - list of ndarray
          : X: the batch - ndarray
          : labels: the labels corresponding to the batch
    Outputs: loss: the negative log-likelihood - float
           : accuracy: the ratio of examples that are well-classified - float
    """ 
    ### Forward propagation
    z = forward(W,b,X)
 
    ### Compute the softmax and the prediction
    out = softmax(z)
    pred = np.argmax(out,axis=1)
    true_label=np.argmax(labels,axis=1)
    
    ### Compute the accuracy
    accuracy =np.mean(pred==true_label)
      
    return accuracy

## Preparing training

The following hyperparameters are given. Next, we can assemble all the function previously defined to implement a training loop. We will train the classifier on **one epoch**, meaning that the model will see each trainin example once. 

In [48]:
# Optimization
eta = 0.01
regularizer = 'L2'
weight_decay = 0.0001

# Training
log_interval = 5000


In [49]:
# Data structures for plotting
g_train_acc=[]
g_valid_acc=[]

#######################
### Learning process ##
#######################
for j in range(n_training):
    ### Getting the example
    X, y = train_images[j].reshape((1,-1)),y_train[j].reshape((1,-1))
    #X, y = train_images[j].reshape((-1,1)),y_train[j].reshape((-1,1))
    ### Forward propagation
    z = forward(W,b,X)

    ### Compute the softmax
    out = softmax(z)
        
    ### Compute the gradient at the top layer
    derror = out - y # This is o - y 

    ### Update the parameters
    W, b = update(eta,W,b,derror,X,regularizer,weight_decay)

    if j % log_interval == 0:
        ### Every log_interval examples, look at the training accuracy
        train_accuracy = computeAcc(W, b, train_images, y_train) 

        ### And the testing accuracy
        test_accuracy = computeAcc(W, b, X_test, y_test) 

        g_train_acc.append(train_accuracy)
        g_valid_acc.append(test_accuracy)
        result_line = str(int(j)) + " train_acc=  " + str(train_accuracy) + " test_acc=  " + str(test_accuracy) + " eta=  " + str(eta)
        print(result_line)

g_train_acc.append(train_accuracy)
g_valid_acc.append(test_accuracy)
result_line = "Final result: " + " train_ac= " + str(train_accuracy) + " test_acc= " + str(test_accuracy) + " eta=  " + str(eta)
print(result_line)     

0 train_acc=  0.09198333333333333 test_acc=  0.098 eta=  0.01
5000 train_acc=  0.8115 test_acc=  0.0982 eta=  0.01
10000 train_acc=  0.8461833333333333 test_acc=  0.0985 eta=  0.01
15000 train_acc=  0.8296833333333333 test_acc=  0.1077 eta=  0.01
20000 train_acc=  0.8419166666666666 test_acc=  0.1124 eta=  0.01
25000 train_acc=  0.8201 test_acc=  0.1121 eta=  0.01
30000 train_acc=  0.7974666666666667 test_acc=  0.104 eta=  0.01
35000 train_acc=  0.7714 test_acc=  0.1028 eta=  0.01
40000 train_acc=  0.82195 test_acc=  0.1006 eta=  0.01
45000 train_acc=  0.8514166666666667 test_acc=  0.1168 eta=  0.01
50000 train_acc=  0.8342666666666667 test_acc=  0.1028 eta=  0.01
55000 train_acc=  0.801 test_acc=  0.1071 eta=  0.01
Final result:  train_ac= 0.801 test_acc= 0.1071 eta=  0.01


What can you say about the performance of this simple linear classifier ? 

We have almost the same behaviour even if the train accuracy differ a little bit from the test_accuracy. We have an acceptable performance.The disadvantage of this linear classifier is that, if the data are strongly non-Gaussian, it can perform quite poorly relative to nonlinear classifiers